I have a couple things to do here.  MLST and trees.  First I need to load in the files from the previous notebook `200819_spades_ill_only.ipynb`

Note - I lost a commit here due to a hard drive crash.  Essentially I had "fixed" some of the isolates that were incorrectly labeled.  

We ended up dropping encl_86R, encl_449 because they are getting flagged as k.pneumo

We also dropped klpn_pilon_613 and klpn_pilon_600 because they are K. quasipneumoniae - not true Kpn. To drop these, I copied the actual gff_encl to gff_full_encl, then deleted the non-clustered assemblies from the gff_encl. Same for gff_klpn.



In [9]:
import pandas as pd
import glob
import os
import re

In [2]:
%cd /atium/Data/Nanopore/cpowgs/trees/200811_timp_tree

/atium/Data/Nanopore/cpowgs/trees/200811_timp_tree


In [3]:
klpn=pd.read_csv("~/Dropbox/timplab_data/cpowgs/200907_tree/klpn.csv", keep_default_na=False)
encl=pd.read_csv("~/Dropbox/timplab_data/cpowgs/200907_tree/encl.csv", keep_default_na=False)
esco=pd.read_csv("~/Dropbox/timplab_data/cpowgs/200907_tree/esco.csv", keep_default_na=False)

Ok - let's gather all the fasta

In [4]:
def move_assemblies(meta, label):
    """This function collects the assemblies into directories with proper labels"""
    for index, sample in meta.iterrows():
        if sample.pilon:
            !cp {sample.pilon} {label}/{label}_pilon_{sample.id}.fasta
        else:
            if sample.spades:
                !cp {sample.spades} {label}/{label}_spades_{sample.id}.fasta


In [17]:
move_assemblies(klpn, label='klpn')
move_assemblies(encl, label='encl')
move_assemblies(esco, label='esco')

Next . . . MLST? How did I do that previously? Found it.

In [19]:
!mlst encl/*fasta --quiet --threads 8 --scheme ecloacae >encl_mlst.tsv
!mlst esco/*fasta --quiet --threads 8 --scheme ecoli >esco_mlst.tsv
!mlst klpn/*fasta --quiet --threads 8 --scheme kpneumoniae >klpn_mlst.tsv


Clean any "pilon" names in the fasta

In [5]:
!sed -i 's/_pilon//g' esco/*fasta
!sed -i 's/_pilon//g' klpn/*fasta
!sed -i 's/_pilon//g' encl/*fasta

Ok - I need to pull down the genbank files for Enterobacter cloacae: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
E. Coli: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
and K. Pneumo: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz

In [6]:
%mkdir refs
%cd refs
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
!gunzip *gz
%mv *gbff ecoli_ref.gb
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
!gunzip *gz
%mv *gbff encl_ref.gb
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz
!gunzip *gz
%mv *gbff kpn_ref.gb
%cd ..

mkdir: cannot create directory ‘refs’: File exists
/atium/Data/Nanopore/cpowgs/trees/200811_timp_tree/refs
--2020-09-13 08:27:56--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 2607:f220:41e:250::7, 2607:f220:41e:250::12, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3462090 (3.3M) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_genomic.gbff.gz’

GCF_000005845.2_ASM 100%[===================>]   3.30M  15.7MB/s    in 0.2s    

2020-09-13 08:27:56 (15.7 MB/s) - ‘GCF_000005845.2_ASM584v2_genomic.gbff.gz’ saved [3462090/3462090]

--2020-09-13 08:27:57--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov).

Ok - now prokka

In [ ]:
for spec in ['encl', 'esco', 'klpn']:
    %rm -R prok_$spec
    %rm -R gff_$spec
    %mkdir gff_$spec
    filelist=(glob.glob(spec+"/*fasta"))
    for myfile in filelist:
        tag=re.split('[\/\.]', myfile)[1]
        print(tag)
        outy='prok_'+spec+'/'+tag
        ref='refs/'+spec+'_ref.gb'
        !prokka --quiet --protein $ref --outdir $outy --prefix $tag --locustag $tag $myfile 
        %cp {outy}/*gff gff_$spec

rm: cannot remove 'prok_encl': No such file or directory
encl_spades_3


Ok - now let's try roary

Roary says first https://github.com/microgenomics/tutorials/blob/master/pangenome.md
you gotta organize the gff files

In [ ]:
for spec in ['encl', 'esco', 'klpn']:
    !roary -i 95 -f pan_{spec} -e -n -g 100000 -p 11 gff_{spec}/*gff

Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.

Please cite Roary if you use any of the results it produces:
    Andrew J. Page, Carla A. Cummins, Martin Hunt, Vanessa K. Wong, Sandra Reuter, Matthew T. G. Holden, Maria Fookes, Daniel Falush, Jacqueline A. Keane, Julian Parkhill,
	"Roary: Rapid large-scale prokaryote pan genome analysis", Bioinformatics, 2015 Nov 15;31(22):3691-3693
    doi: http://doi.org/10.1093/bioinformatics/btv421
	Pubmed: 26198102

Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.
Use of uninitialized value in require at (eval 15229) line 1.


Counting snps from the multi-fa using snp-sites tool

I used to generate a `vcf` using `snp-sites` - then loaded it into R and calulcated the distance, but I no longer think that's necessary, thanks again to the awesome torst, he wrote a tool to do it directly from the aln.

In [15]:
for spec in ['encl', 'esco', 'klpn']: 
    !snp-dists -c pan_{spec}/core_gene_alignment.aln >pan_{spec}/core_gene_alignment_count.csv

This is snp-dists 0.7.0
Read 80 sequences of length 313503
This is snp-dists 0.7.0
Read 45 sequences of length 1042097
This is snp-dists 0.7.0
Read 120 sequences of length 178733


Ok - that generated a pan genome and core alignment file which is already getting me close.  I then ran it in geneious, but let me take a guess and try to run in raxml
some tips:
-p is a random parsimony seed - to make sure analysis does the same thing
-m GTRGAMMA - GTR GAMMA method
-x is the rapid bootstrapping seed
-N is the number of bootstrapops
-f a says do rapid bootstrap

In [ ]:
for spec in ['encl', 'esco', 'klpn']:
    %rm -R raxml_{spec}
    %mkdir raxml_{spec}
    %cd raxml_{spec}
    !raxmlHPC-PTHREADS -T 11 -f a -m GTRGAMMA -p 53243 -x 72341 -s ../pan_{spec}/core_gene_alignment.aln -N 100 -n testrapid.tree
    %cd ..

rm: cannot remove 'raxml_encl': No such file or directory
/atium/Data/Nanopore/cpowgs/trees/200811_timp_tree/raxml_encl

RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file



Using BFGS method to optimize GTR rate parameters, to disable this specify "--no-bfgs" 


This is the RAxML Master Pthread

This is RAxML Worker Pthread Number: 3

This is RAxML Worker Pthread Number: 5

This is RAxML Worker Pthread Number: 2

This is RAxML Worker Pthread Number: 4

This is RAxML Worker Pthread Number: 6

This is RAxML Worker Pthread Number: 7

This is RAxML Worker Pthread Number: 8

This is RAxML Worker Pthread Number: 10

This is RAxML Worker Pthread Number: 9

This is RAxML Worker Pthread Number: 1


This is RAxML version 8.2.12 released by Alexandros Stamatakis on May 2018.

With greatly appreciated code contributions by:
Andre Aberer      (HITS)
Simon Berger      (HITS)
Alexey Kozlov     (HITS)
Kassian Kobert    (HITS)
David Dao         (KIT and HI